# 200 : Rethink

Approaches from phase 1 does not work at all for the new data. We need to rethink our approach.

## Web References

- [Time Series Forecasting with PyCaret Regression Module](https://towardsdatascience.com/time-series-forecasting-with-pycaret-regression-module-237b703a0c63a)

In [ ]:
import numpy as np
import pandas as pd
import logging

import matplotlib.pyplot as plt
import plotly.express as px

from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error 

## Data Collection

In [ ]:
# load the source training data
df_source = pd.read_csv('../../data/input/df_train.csv')

print(df_source.shape)
with pd.option_context('display.max_columns', None):
    display(df_source.head(3))

In [ ]:
# load the competition data
df_competition = pd.read_csv('../../data/input/df_test.csv')

print(df_source.shape)
with pd.option_context('display.max_columns', None):
    display(df_source.head(3))

## Data Cleaning

In [ ]:
def clean_unused_columns(data:pd.DataFrame) -> pd.DataFrame:
    """
    Remove unused columns from the dataset
    """
    df_clean = data.copy()
    df_clean.drop(columns=['Unnamed: 0'], inplace=True)

    return df_clean

In [ ]:
def clean_datatypes(data:pd.DataFrame) -> pd.DataFrame:
    """
    Set the columns to the correct datatypes
    """
    df_clean = data.copy()
    df_clean['time'] = pd.to_datetime(df_clean['time'])

    return df_clean

In [ ]:
def clean_valencia_pressure(data:pd.DataFrame) -> pd.DataFrame:
    """
    Replace the nulls in Valencia_pressure with the mode
    """
    df_clean = data.copy()

    # forward fill empty values in Valencia_pressure
    df_clean['Valencia_pressure'] = df_clean['Valencia_pressure'] \
        .fillna(method='ffill')

    return df_clean

In [ ]:
def clean_valencia_wind(data:pd.DataFrame) -> pd.DataFrame:
    """
    Clean Valencia wind degrees by striping string & convert to numerical
    """
    df_clean = data.copy()

    df_clean['Valencia_wind_deg'] = df_clean['Valencia_wind_deg'].str.extract('(\\d+)')
    df_clean['Valencia_wind_deg'] = pd.to_numeric(df_clean['Valencia_wind_deg'])

    return df_clean

In [ ]:
def clean_seville_pressure(data:pd.DataFrame) -> pd.DataFrame:
    """
    Remove non-numeric values
    """
    df_clean = data.copy()

    df_clean['Seville_pressure'] = df_clean['Seville_pressure'] \
        .str.extract('(\\d+)') \
        .astype(float)

    return df_clean

In [ ]:
def clean_data(dat:pd.DataFrame) -> pd.DataFrame:
    """
    Clean the data
    """
    df_clean = dat.copy()

    df_clean = clean_datatypes(df_clean)
    df_clean = clean_unused_columns(df_clean)

    df_clean = clean_valencia_pressure(df_clean)
    df_clean = clean_valencia_wind(df_clean)
    df_clean = clean_seville_pressure(df_clean)

    return df_clean   

## Exploratory Data Analysis

In [ ]:
# create a cleaned dataset to explore
df_explore = clean_data(df_source)

The dataset contains time series data, so we need to investigate if there are any tends and seasonality in the data.

We will start by looking at the data summarized (mean) by month to see the big picture.

In [ ]:
# Resample the data to daily frequency
df_train_resampled = df_explore.resample('M', on='time').mean()

# Group the DataFrame by year
year_groups = df_train_resampled.groupby(df_train_resampled.index.year)

# # Create subplots for each year
# fig, axs = plt.subplots(len(year_groups), 1, figsize=(15, 5.5*len(year_groups)))
# for i, (year, data) in enumerate(year_groups):
#     axs[i].plot(data.index, data['load_shortfall_3h'])
#     axs[i].set_xlabel('Time')
#     axs[i].set_ylabel('Load Shortfall 3h')
#     axs[i].set_title(f'Load Shortfall 3h for {year}')

# plt.tight_layout()
# plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
for i, (year, data) in enumerate(year_groups):
    ax.plot(data.index - pd.DateOffset(years=i), data['load_shortfall_3h'], label=str(year))

ax.set_xlabel('Time')
ax.set_ylabel('Load Shortfall 3h')
ax.set_title('Load Shortfall 3h for each year')
ax.legend()

plt.show()

We can see that the data is not stationary, there is a clear trend and seasonality.

- The **2016** amd **2017** data track each other well and is a good example of showing that there is clear seasonality in the data.


- The throughout the year the **2016** data is lower by about the same amount compared to the **2017** data indicating that we might expect a year-on-year increase in the shortfall. This shows that there could be a clear trend in the data.

- The **2015** data is following a similar seasonal pattern, but is not tracking the **2016** and **2017** data as well. The trend also does not match with what we see in the other years. We have to be careful when using this data as it might not be representative of the **2018** data we hope to predict. 

- There could be factors causing the data for **2015** to look different, we could experiment with only using the **2016** and **2017** data for training to see if we can get better results.

- Since the data is not stationary with clear trend and seasonality. We will need experiment with removing these to make the data stationary for best results.

- We will probably have to be cautious with the sharp drop in the **2017** data in the last month. We can experiment with excluding this month to see if we can get better results.


## Feature Engineering

In [ ]:
df_temp = clean_datatypes(df_source)

In [ ]:
def add_date_features(data:pd.DataFrame) -> pd.DataFrame:
    """
    Split the date into its separate parts (years, mont, etc.)
    """
    df_clean = data.copy()

    df_clean['Year'] = df_clean['time'].dt.year
    df_clean['Month'] = df_clean['time'].dt.month
    df_clean['Day'] = df_clean['time'].dt.day
    df_clean['Hour'] = df_clean['time'].dt.hour
    df_clean['Day_of_week'] = df_clean['time'].dt.dayofweek
    df_clean['Day_of_year'] = df_clean['time'].dt.dayofyear

    return df_clean

In [ ]:
def add_season_feature(data:pd.DataFrame) -> pd.DataFrame:
    """
    Add a season feature based on the month.
    """
    df_clean = data.copy()

    # coding for the seasons
    season = {
            12:'Winter', 1:'Winter', 2:'Winter',
            3:'Spring', 4:'Spring', 5:'Spring',
            6:'Summer', 7:'Summer', 8:'Summer',
            9:'Autumn', 10:'Autumn', 11:'Autumn'}
    
    # add a season column based on the mapping
    df_clean['Season'] = df_clean.time.dt.month.map(season)

    return df_clean

In [ ]:
def add_day_features(data:pd.DataFrame) -> pd.DataFrame:
    """
    Add a feature to indicate if it is a weekday
    """
    df_clean = data.copy()
    day_type = []

    day_mapping = {
        0: 'Weekday', 1: 'Weekday', 2: 'Weekday', 3: 'Weekday',
        4: 'Weekday', 5: 'Weekend', 6: 'Weekend'}

    # add a day type based on the mapping
    df_clean['day_type'] = df_clean.time.dt.day_of_week.map(day_mapping)

    return df_clean

In [ ]:
def add_lag_feature(data:pd.DataFrame, column:str, lags:int) -> pd.DataFrame:
    """
    Add lag features to the dataset
    """
    df_clean = data.copy()

    for lag in range(1, lags + 1):
        df_clean[f'{column}_lag_{lag}'] = df_clean[column].shift(lag)

    return df_clean

In [ ]:
def add_lag_features(data:pd.DataFrame, lags:int) -> pd.DataFrame:
    """Add lag features to the dataset"""

    df_features = data.copy()
    cities = ['Madrid', 'Valencia', 'Seville', 'Bilbao', 'Barcelona']

    measures = [
        'wind_speed',
        'wind_deg',
        'rain_1h',
        'rain_3h',
        'humidity',
        'clouds_all',
        'pressure',
        'snow_3h',
        'weather_id',
        'temp_max',
        'temp_min',
        'temp'
    ]

    for city in cities:
        for feature in measures:
            if f'{city}_{feature}' in df_features.columns:
                df_features = add_lag_feature(df_features, f'{city}_{feature}', lags)
            else:
                logging.warn(f'{city}_{feature} not in dataset')

    return df_features

In [ ]:
def add_rolling_average_features(data:pd.DataFrame, window:int) -> pd.DataFrame:
    """
    Add rolling average features to the dataset
    """
    df_features = data.copy()
    cities = ['Madrid', 'Valencia', 'Seville', 'Bilbao', 'Barcelona']

    measures = [
        'wind_speed',
        'wind_deg',
        'rain_1h',
        'rain_3h',
        'humidity',
        'clouds_all',
        'pressure',
        'snow_3h',
        'weather_id',
        'temp_max',
        'temp_min',
        'temp'
    ]

    for city in cities:
        for feature in measures:
            if f'{city}_{feature}' in df_features.columns:
                df_features[f'{city}_{feature}_rolling_avg_{window}'] = df_features[f'{city}_{feature}'].rolling(window=window).mean()
            else:
                logging.warn(f'{city}_{feature} not in dataset')

    return df_features


In [ ]:
def feature_engineering(data:pd.DataFrame) -> pd.DataFrame:
    """
    Clean the dataset and add the features
    """
    df_clean = data.copy()

    # define categorical columns
    categorical_columns = [
        'Month', 'Day', 'Hour', 
        'Day_of_week', 'Season', 'Day_type'
    ]

    # perform cleaning and feature engineering
    df_clean = clean_data(df_clean)
    df_clean = add_date_features(df_clean)
    df_clean = add_season_feature(df_clean)
    df_clean = add_day_features(df_clean)

    # # add rolling average features
    # df_clean = add_rolling_average_features(df_clean, 5)
    # df_clean = add_rolling_average_features(df_clean, 10)
    # df_clean = add_rolling_average_features(df_clean, 30)
    # df_clean = add_rolling_average_features(df_clean, 60)
    # df_clean = add_rolling_average_features(df_clean, 90)
    # df_clean = add_rolling_average_features(df_clean, 365)


    # # add lag features
    # df_clean = add_lag_features(df_clean, 10)
    
    # replace empty lag values with 0
    df_clean.fillna(0, inplace=True)

    return pd.get_dummies(
        df_clean, 
        drop_first=True)
    

    # return pd.get_dummies(
    #     df_clean, 
    #     columns=categorical_columns,
    #     drop_first=True)

# clean the dataset and add new features
df_features = feature_engineering(df_source)

with pd.option_context('display.max_columns', None):
    display(df_features.head(3))

## Looking for Trends and Seasonality

Lets see what the data looks like with a moving average.

In [ ]:
# get the data for plotting
df_plot = feature_engineering(df_source)

# create a moving average
df_plot['moving_average'] = df_plot['load_shortfall_3h'].rolling(8*30).mean()

# plot the data and moving average
fig = px.line(df_plot, x="time", y=["load_shortfall_3h", "moving_average"])
fig.show()

The Data has so much variation in it that will make prediction very hard. Lets try and zoom in a bit and look at a single month.

In [ ]:
# plot data for a single month
fig = px.line(
    df_plot[(df_plot['time'] >= '2016-01-01') & (df_plot['time'] < '2016-02-01')],
    x="time", 
    y=["load_shortfall_3h", 
    "moving_average"])

fig.show()

Let's get down to day level and see what the data looks like.

In [ ]:
def plot_days(data:pd.DataFrame, start_date: str, end_date: str) -> None:
    """
    Plot each day in the given date range over the top of each other.
    """
    df = data[(data['time'] >= start_date) & (data['time'] <= end_date)]

    # Create a new figure object
    fig, ax = plt.subplots()

    # Loop over each day and add a line to the plot for that day
    for i, day in enumerate(pd.date_range(start=start_date, end=end_date, freq='D')):
        # Filter the data for the current day
        day_data = df[df['time'].dt.date == day.date()]
        
        # Plot the data for the current day on the same y-axis
        x_values = [t.strftime('%H:%M:%S') for t in day_data['time'].dt.time]
        
        ax.plot(
            x_values, 
            day_data['load_shortfall_3h'],
            label=day.date())

    # Set the y-axis label and title
    ax.set_ylabel('Load Shortfall 3h')
    ax.set_title('Load Shortfall 3h for each day from {} to {}'.format(start_date, end_date))

    # Add a legend
    ax.legend()

    # Show the plot
    plt.show()

In [ ]:
print('January')
plot_days(df_plot, '2015-01-10', '2015-01-15')
plot_days(df_plot, '2016-01-10', '2016-01-15')
plot_days(df_plot, '2017-01-10', '2017-01-15')

print('June')
plot_days(df_plot, '2015-06-20', '2015-06-25')
plot_days(df_plot, '2016-06-20', '2016-06-25')
plot_days(df_plot, '2017-06-20', '2017-06-25')

## Model Training

#### Model Evaluation Functions

In [ ]:
evaluation_results = []

In [ ]:
def show_evaluation(y_test, y_predict):
    # calculate the metrics
    mae = mean_absolute_error(y_test, y_predict) 
    mse = mean_squared_error(y_test, y_predict) 
    rmse = np.sqrt(mse) 
    r2 = r2_score(y_test, y_predict)

    print(f'RMSE: {rmse:.3f} | R-squared: {r2:.4f}')

    return rmse, r2

### Prophet

In [ ]:
def create_prophet_model(data:pd.DataFrame):
    """
    Create and fit a Prophet model that can be used to predict future
    data.
    """
    # perform feature engineering
    df_train = feature_engineering(data)

    # rename the columns to what prhophet expects
    df_train.rename(columns={'load_shortfall_3h': 'y', 'time': 'ds'}, inplace=True)

    # create the prophet model
    model = Prophet(
        seasonality_mode='multiplicative',
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=0.1,
        interval_width=0.8,
        yearly_seasonality=True)

    # fit the model to the training data
    model.fit(df_train)

    # return the model
    return model

In [ ]:
def get_prophet_predictions(model, data:pd.DataFrame):
    """
    Get predictions from the Prophet model
    """
    df_test = data.copy()

    # rename the columns
    df_test.rename(columns={'load_shortfall_3h': 'y', 'time': 'ds'}, inplace=True)

    # make predictions for the future dates
    forecast = model.predict(df_test)

    # extract the relevant columns from the forecast
    predictions = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

    return predictions


### All Years

In [ ]:
# create the model
prophet_model = create_prophet_model(df_source)

# get the competition data
df_test = feature_engineering(df_competition)

# get the predictions
predictions = get_prophet_predictions(prophet_model, df_test)

In [ ]:
# plot the predictions
predictions.plot(x='ds', y='yhat', kind='line', figsize=(15, 5))

Save the data so we can test it with a Kaggel submission.

In [ ]:
df_out = df_test \
    .rename(columns={'time': 'ds'})[['ds']] \
    .merge(predictions[['ds', 'yhat']], on='ds', how='left') \
    .rename(columns={'ds': 'time'}) \
    .rename(columns={'yhat': 'load_shortfall_3h'})

display(df_out)

#Output to csv for Kaggle 
df_out.to_csv("../../data/output/200_all_prophet.csv",index=False)

We get a score of`4070.06422` on the public leaderboard. This is not a good good result and puts as at position `74` out of `141` teams.

### Latest Two Years

We have seen in our analysis that the **2015** data is potentially not representative of future years, so we will try and build a model without it.

In [ ]:
# select last two years
df_train = df_source[df_features['time'] >= '2016-01-01']

# create the model
prophet_model = create_prophet_model(df_train)

# get the competition data
df_test = feature_engineering(df_competition)

# get the predictions
predictions = get_prophet_predictions(prophet_model, df_test)

# plot the predictions
predictions.plot(x='ds', y='yhat', kind='line', figsize=(15, 5))

This plot looks very similar to the plot we saw when we used all the data. This might be an indication that we might not get a better result.

In [ ]:
df_out = df_test \
    .rename(columns={'time': 'ds'})[['ds']] \
    .merge(predictions[['ds', 'yhat']], on='ds', how='left') \
    .rename(columns={'ds': 'time'}) \
    .rename(columns={'yhat': 'load_shortfall_3h'})

display(df_out)

#Output to csv for Kaggle 
df_out.to_csv("../../data/output/200_last_two_prophet.csv",index=False)

Excluding **2015** data results in a worse score of `4275.24288` on the public leaderboard.

As a final try excluding the last month of **2017** data.

In [ ]:
# select last two years, but exclude the last month
df_train = df_source[
    (df_features['time'] >= '2016-01-01') & \
    (df_features['time'] < '2017-12-01')
]

# create the model
prophet_model = create_prophet_model(df_train)

# get the competition data
df_test = feature_engineering(df_competition)

# get the predictions
predictions = get_prophet_predictions(prophet_model, df_test)

# plot the predictions
predictions.plot(x='ds', y='yhat', kind='line', figsize=(15, 5))

In [ ]:
df_out = df_test \
    .rename(columns={'time': 'ds'})[['ds']] \
    .merge(predictions[['ds', 'yhat']], on='ds', how='left') \
    .rename(columns={'ds': 'time'}) \
    .rename(columns={'yhat': 'load_shortfall_3h'})

display(df_out)

#Output to csv for Kaggle 
df_out.to_csv("../../data/output/200_last_two_sans_12_prophet.csv",index=False)

The predictions for the last month is higher now. But this has drastically reduced the score to `5135.65675` on the public leaderboard.

There is however an interesting idea here that we could try. We could predict the entire **2018** dataset with a baseline model, the use a second model to predict a selected month only and replace this in the base model predicts, then see what influence this has on the Kaggle score.

## Pycaret

Lets see what the data looks like with a moving average.

In [ ]:
# get the data for plotting
df_plot = feature_engineering(df_source)

# create a moving average
df_plot['moving_average'] = df_plot['load_shortfall_3h'].rolling(8*30).mean()

# plot the data and moving average
fig = px.line(df_plot, x="time", y=["load_shortfall_3h", "moving_average"])
fig.show()

The Data has so much variation in it that will make prediction very hard. Lets try and zoom in a bit and look at a single month.

In [ ]:
# plot data for a single month
fig = px.line(
    df_plot[(df_plot['time'] >= '2016-01-01') & (df_plot['time'] < '2016-02-01')],
    x="time", 
    y=["load_shortfall_3h", 
    "moving_average"])

fig.show()

Let's get down to day level and see what the data looks like.

In [ ]:
def plot_days(data:pd.DataFrame, start_date: str, end_date: str) -> None:
    """
    Plot each day in the given date range over the top of each other.
    """
    df = data[(data['time'] >= start_date) & (data['time'] <= end_date)]

    # Create a new figure object
    fig, ax = plt.subplots()

    # Loop over each day and add a line to the plot for that day
    for i, day in enumerate(pd.date_range(start=start_date, end=end_date, freq='D')):
        # Filter the data for the current day
        day_data = df[df['time'].dt.date == day.date()]
        
        # Plot the data for the current day on the same y-axis
        x_values = [t.strftime('%H:%M:%S') for t in day_data['time'].dt.time]
        
        ax.plot(
            x_values, 
            day_data['load_shortfall_3h'],
            label=day.date())

    # Set the y-axis label and title
    ax.set_ylabel('Load Shortfall 3h')
    ax.set_title('Load Shortfall 3h for each day from {} to {}'.format(start_date, end_date))

    # Add a legend
    ax.legend()

    # Show the plot
    plt.show()

In [ ]:
print('January')
plot_days(df_plot, '2015-01-10', '2015-01-15')
plot_days(df_plot, '2016-01-10', '2016-01-15')
plot_days(df_plot, '2017-01-10', '2017-01-15')

print('June')
plot_days(df_plot, '2015-06-20', '2015-06-25')
plot_days(df_plot, '2016-06-20', '2016-06-25')
plot_days(df_plot, '2017-06-20', '2017-06-25')

- The seasonality is down to the lowest level in the data, namely the 3 hour period.

- As a simple experiment we can try and do predictions using the hour as the only feature, but this is what the Prophet is essentially already doing by only using the date as a feature.

- We could add a day of year feature and test if it helps.

## Extra Trees